In [ ]:
 from PIL import Image, ImageFilter
import numpy as np
import random
import copy, matplotlib.pyplot as plt
from keras.utils import Progbar
import itertools
import cmath
import pandas as pd, pickle
from IPython.display import clear_output

# Impliment Daniel Borowski and Mike Pound's implimetation of Node based A* search

In [ ]:
import astar, mazes

# Open the Image of the road from <html>snazzymaps.com</html>

In [ ]:
im=Image.open("Map.bmp")
im = im.convert('L')
im

# At current image zoom level, A pixel is approximately $7\frac{3}{4}$ meters<br> 129 pixels is a kilometer <br> 208 pixels in a mile

if Truck moves at 30 MPH => 30*208 PixPH or 104 Pix/Minute $\therefore$ 1 Minute == 104 Pix

In [ ]:

def Get_Address(im,Numb):
    """
    Get a Randomly Selected set of addresses
    """
    ara=np.array(im)

    T=50

    alll=np.all([ara[1:-1,1:-1]<T,
                                                              np.any([
                                                                  ara[ :-2,1:-1]>T,
                                                                  ara[ 2: ,1:-1]>T,
                                                                  ara[1:-1, :-2]>T,
                                                                  ara[1:-1,2:  ]>T
                                                              ],axis=0)],axis=0)
    
    return random.choices(list(zip(*np.where(alll)+np.array([[1,1] for pairs in range(np.where(alll)[0].shape[0])]).T  )           )
                          ,k=Numb)
Get_Address(im,5)

In [ ]:
maze=mazes.Maze(im)

## Proof Of Concept Random-To-Random Solve

### Path with breaks of straights

### Distance Traveled (in Pixels)

## Implementation of a $\Theta$($n^2$*$2^n$) complexity Traveling Salesman Solution 
>Compare to $\Theta$(n!) Naïve Full Permutation search

In [ ]:

def travelling(graph, a, b, mi,PA,SSS):
        
            if(len(a)==0):
                return graph[b][0],[b]
            for i in range(len(a)):
                SSS.add(1)
#                 print(int(a[i]))
                ReS,ReP=travelling(graph,a[0:i]+a[i+1:],int(a[i]),mi,PA,SSS)
#                 print(ReS)
#                 print(ReP)
                ReS+=graph[b][int(a[i])]
                if ReS<mi:
                    mi=ReS
                    ReP.append(b)
                    PA=ReP

            return mi,PA




## Implimentation of a Greedy-Linkage $\Theta(n^3)$ Approximate Solution 

In [ ]:
def shortest_edges_first(Grid):
    itt=np.nditer(Grid,flags=['multi_index'])
    arralist=[]
    # keylist=[]
    while not itt.finished:
        if itt.multi_index[0]>itt.multi_index[1]:
            arralist.append([itt.multi_index,itt[0]])
    #         keylist.append(itt[0])
        itt.iternext()
    return [x for x,_ in sorted(arralist,key=lambda inst: inst[1])]

def join_endpoints(endpoints, A, B):
    "Join B's segment onto the end of A's and return the segment. Maintain endpoints dict."
    Asegment, Bsegment = endpoints[A], endpoints[B]
    if Asegment[-1] is not A: Asegment.reverse()
    if Bsegment[0] is not B: Bsegment.reverse()
    Asegment.extend(Bsegment)
    del endpoints[A], endpoints[B] # A and B are no longer endpoints
    endpoints[Asegment[0]] = endpoints[Asegment[-1]] = Asegment
    return Asegment

def greedy_tsp(graph, a, b, mi,PA,SSS):
    """Go through edges, shortest first. Use edge to join segments if possible."""
    endpoints = {c: [c] for c in range(graph.shape[0])} # A dict of {endpoint: segment}
    for (A, B) in shortest_edges_first(graph):
        if A in endpoints and B in endpoints and endpoints[A] != endpoints[B]:
            new_segment = join_endpoints(endpoints, A, B)
            if len(new_segment) == graph.shape[0]:
#                 SSS.add(1)
                return new_segment
            
def alter_tour(tour,Grid):
    "Try to alter tour for the better by reversing segments."
    original_length = tour_length(tour,Grid)
    for (start, end) in all_segments(len(tour)):
        reverse_segment_if_better(tour, start, end,Grid)
    # If we made an improvement, then try again; else stop and return tour.
    if tour_length(tour,Grid) < original_length:
        return alter_tour(tour,Grid)
    return tour

def all_segments(N):
    "Return (start, end) pairs of indexes that form segments of tour of length N."
    return [(start, start + length)
            for length in range(N, 2-1, -1)
            for start in range(N - length + 1)]

def reverse_segment_if_better(tour, i, j,Grid):
    "If reversing tour[i:j] would make the tour shorter, then do it." 
    # Given tour [...A-B...C-D...], consider reversing B...C to get [...A-C...B-D...]
    A, B, C, D = tour[i-1], tour[i], tour[j-1], tour[j % len(tour)]
    # Are old edges (AB + CD) longer than new ones (AC + BD)? If so, reverse segment.
    if Grid[A][B] + Grid[C][D] > Grid[A][C] + Grid[B][D]:
        tour[i:j] = reversed(tour[i:j])

def tour_length(tour,Grid):
    "The total of distances between each pair of consecutive cities in the tour."
    return sum(Grid[tour[i]][tour[i-1]] 
               for i in range(len(tour)))

def fix_tour(tour):
    return (tour[tour.index(0):]+tour[:tour.index(0)])[1:]+[0]

def fixed_alternate_greedy(graph, a, b, mi,PA,SSS):
    tour=fix_tour(alter_tour(greedy_tsp(graph, a, b, mi,PA,SSS),graph))
    length=tour_length(tour,graph)
    return length,tour

## Useful Functions for Model buidling

In [ ]:

def Nearest(t,L,exclude=[],D=100000):
    """
    Finds the point in L nearest to t without moving in any direction not towards exclude so long as it is Within distance D
    """
#     print(t)
    if D>0:
        TT=[]
        for k in t:
            for move in [[i,j] for i in [0,1,-1] for j in [0,1,-1] if abs(i)+abs(j)==1 and [i,j] not in exclude]:
#                 print(k)
#                 print(move)
                tri=(k[0]+move[0],k[1]+move[1])
                if tri in L:
                    return tri
                TT.append(tri)
        TT=list(set(TT))
        return Nearest(TT,L,exclude,D-1)

def Nodes_Near(where,out,ma,MapL,k,Drange=10000):
    """
    Finds the set of drop off locations for Situatiation 2 Drone-Complient Model
    """
    temp=[(np.array(where)+np.array(k)).round(0) for k in [(out*np.cos(np.pi*2 * theta/ma),out*np.sin(np.pi*2 * theta/ma)) for theta in range(ma)]]
    temp=np.array(temp,dtype=np.int)
    temp={tuple(k) for k in temp}
    Final=set()
    for t in temp:
        k.add(1)
        if t in MapL:
            Final.add(t)
        else:
            te=Nearest([t],MapL,[[where[0]-t[0],where[1]-t[1]]],Drange)
            if te:
                Final.add(te)
    if Final==set():
        Final.add(Nearest([where],MapL))
    
    return Final



def Nodes_Near_LEGACY(where,out,ma,MapL,k,Drange=10000):
    """
    Finds the set of drop off locations for Situation 3 Fully Autonomous Model
    """
    Final=set()
    for outt in range(int(round(np.log(out)/np.log(4))+1)):
        out=pow(4,outt)
        temp=[(np.array(where)+np.array(k)).round(0) for k in [(out*np.cos(np.pi*2 * theta/ma),out*np.sin(np.pi*2 * theta/ma)) for theta in range(ma)]]
        temp=np.array(temp,dtype=np.int)
        temp={tuple(k) for k in temp}

        for t in temp:
            k.add(1)
            if t in MapL and mindist(t,Final)>4:
                Final.add(t)
            else:
                te=Nearest([t],MapL,[[where[0]-t[0],where[1]-t[1]]],Drange)
                if te:
                    if mindist(te,Final)>4:
                        Final.add(te)
    if Final==set():
        Final.add(Nearest([where],MapL))
    return Final

def mindist(te,Final):
    """
    Computes the minimum euclidian distance between point te and points in list Final
    """
    maxd=np.inf
    a=np.array(te)
    for par in Final:
        b=np.array(par)
        dist=np.linalg.norm(a-b)
        if dist<maxd:
            maxd=dist
    return maxd
# Nodes_Near((90,130),4,6,maze.NodeList)

In [ ]:
class Traditional_Model():
    """
    Situation 1
    called by both Situation 2 and 3
    """
    def __init__(self,Map,Address,T_Speed,Driver_Delay,k1=None,SSS=None,verbose=False,Path_Grid=None):
        if Path_Grid is not None:
            self.Path_Grid=Path_Grid
        else:
            if not k1:
                pass
                k1=Progbar(target=round(.5*pow(len(Address),2))+len(Address))
            if not SSS:
                SSS=Progbar(target=np.math.factorial(len(Address)))
            self.Path_Grid=np.empty((len(Address),len(Address)),dtype=np.object)

            for A in range(len(Address)):
                for B in range(A,len(Address)):
                    if verbose:k1.add(1)
                    Map.start=Map.NodeList[Nearest([Address[A]],Map.NodeList)]
                    Map.end  =Map.NodeList[Nearest([Address[B]],Map.NodeList)]
                    x=astar.solve(Map)[0]
                    xc=[]
                    for eac in range(len(x)-1):
                        for (X,Y) in [(X,Y) for X in list(range(x[eac].Position[0], x[eac+1].Position[0], 
                                                    [k if k==-1 else 1 for k in [np.sign(x[eac+1].Position[0]-x[eac].Position[0])]][0]
                                                     ))+[x[eac+1].Position[0]] 
                                     for Y in list(range(x[eac].Position[1], x[eac+1].Position[1], 
                                                    [k if k==-1 else 1 for k in [np.sign(x[eac+1].Position[1]-x[eac].Position[1])]][0]
                                                     ))+[x[eac+1].Position[1]]
                                    ][:-1]:
                            xc.append((X,Y))
                    x=xc
                    self.Path_Grid[A][B]=x
                    x=x.copy()
                    x.reverse()
                    self.Path_Grid[B][A]=x
            if verbose: k1.update(k1.target)
#         print("Solved Paths")
        self.Travel_Grid=np.array([[
            len(x)    
            for x in y] for y in self.Path_Grid])
#         print("Solving Traveling")
        
        if len(Address)<4:
            usefunc=travelling
        else:
            usefunc=fixed_alternate_greedy
    
        self.Travel=usefunc(self.Travel_Grid,[k for k in range(1,len(Address))],0,np.inf,[0],SSS)
        Travel=list(self.Travel)
        Travel[0]=complex(Travel[0],(len(Address)-1)*Driver_Delay)
        self.Travel=tuple(Travel)
#         SSS.update(SSS.target)
    
    def fullPath(self):
        Travel=[0]+self.Travel[1]
        print(Travel)
        return np.concatenate([self.Path_Grid[Travel[i]][Travel[i+1]] for i in range(len(Travel)-1) if self.Path_Grid[Travel[i]][Travel[i+1]]!=[]])

In [ ]:
im2=im.filter(ImageFilter.BoxBlur(30))

In [ ]:
class Drone_Model_FAA_Comp():
    """
    Situation 2
    """
    def __init__(self,Map,Address,T_speed,D_Range,D_speed,D_number,Driver_Delay):
        self.Address=Address
        self.T_speed=T_speed
        self.D_Range=D_Range
        self.D_speed=D_speed
        self.D_number=D_number
        self.Travel=(complex(np.inf,0),[])
        k=Progbar(target=2*(len(Address))*len(range(0,D_Range,2)))
        self.Drone_Drop=[Nodes_Near(Addres,4,4,Map.NodeList,k,D_Range) if Addres not in zip(*np.where(np.array(im2)>12))
                         else [Nearest([Addres],Map.NodeList)] for Addres in Address[1:]]
#         print(self.Drone_Drop)
        
        Paths=dict()
        print("")
        print("Building All Paths before Runtime")
        k1=Progbar(target=sum([len(list(itertools.product(*comb))) for comb in itertools.combinations([set(Address[0:1])]+self.Drone_Drop,2)]))
        
        for startstop in itertools.combinations([set(Address[0:1])]+self.Drone_Drop,2):
            for start,stop in itertools.product(*startstop):
                    k1.add(1)
                    Map.start=Map.NodeList[Nearest([start],Map.NodeList)]
                    Map.end  =Map.NodeList[Nearest([stop],Map.NodeList)]
                    x=astar.solve(Map)[0]
                    xc=[]
                    for eac in range(len(x)-1):
                        for (X,Y) in [(X,Y) for X in list(range(x[eac].Position[0], x[eac+1].Position[0], 
                                                    [k if k==-1 else 1 for k in [np.sign(x[eac+1].Position[0]-x[eac].Position[0])]][0]
                                                     ))+[x[eac+1].Position[0]] 
                                     for Y in list(range(x[eac].Position[1], x[eac+1].Position[1], 
                                                    [k if k==-1 else 1 for k in [np.sign(x[eac+1].Position[1]-x[eac].Position[1])]][0]
                                                     ))+[x[eac+1].Position[1]]
                                    ][:-1]:
                            xc.append((X,Y))
                    x=xc

                    Paths[(start,stop)]=x
                    x=x.copy()
                    x.reverse()

                    Paths[(stop,start)]=x
        
        
        self.Paths=Paths
        
        
        
        k2=Progbar(target=len(list(itertools.product(*self.Drone_Drop))))
        
        for path in itertools.product(*self.Drone_Drop):
            
            path=tuple(Address[0:1])+path
            
            PathGrid=np.array([[Paths[(begin,end)] if begin!=end else [begin] for begin in path] for end in path])
    
    
            self.pat=Traditional_Model(Map,path,T_speed,Driver_Delay/2,k1,k2,False,Path_Grid=PathGrid)
            self.pat.Travel=(self.pat.Travel[0],[0]+self.pat.Travel[1])
            
            full=self.pat.Path_Grid[self.pat.Travel[1][0]][self.pat.Travel[1][1]]
            
            for each in range(len(self.pat.Travel[1])-2):
                self.pat.Travel=(complex(self.pat.Travel[0].real,self.pat.Travel[0].imag+2*T_speed/D_speed*np.sqrt(pow(Address[self.pat.Travel[1][each+1]][0]-path[self.pat.Travel[1][each+1]][0],2) + pow(Address[self.pat.Travel[1][each+1]][1]-path[self.pat.Travel[1][each+1]][1],2))),
                                 self.pat.Travel[1])
                
                full=full+[path[self.pat.Travel[1][each+1]] for xx in range(int(2*T_speed/D_speed*np.sqrt(pow(Address[self.pat.Travel[1][each+1]][0]-path[self.pat.Travel[1][each+1]][0],2) + pow(Address[self.pat.Travel[1][each+1]][1]-path[self.pat.Travel[1][each+1]][1],2))))]
                full=full+self.pat.Path_Grid[self.pat.Travel[1][each+1]][self.pat.Travel[1][each+2]]
            
            
            if self.pat.Travel[0].real+self.pat.Travel[0].imag<self.Travel[0].real+self.Travel[0].imag:
                self.Travel=self.pat.Travel
                self.path=self.pat
                self.points=path
                self.full=full
            k2.add(1)
        k2.update(k2.target)
                
    def fullPath(self):
        print(len(self.Travel[1])-1)
        return self.full

# Define Subsetting via Minimum Area Convex Polygon

In [ ]:
class Edge:
    def __init__(self,end1,end2):
        # D-X D-Y Intercept
        self.function=[end1[0]-end2[0],end1[1]-end2[1]]
        if end2[0]*self.function[1]-end2[1]*self.function[0]!=end1[0]*self.function[1]-end1[1]*self.function[0]:
            print("{0}*{1}+{2}*{3}!={4}*{1}+{5}*{3}".format(end2[0],self.function[1],end2[1],self.function[0],end1[0],end1[1]))
        else:
            self.function.append(end2[0]*self.function[1]-end2[1]*self.function[0])
        self.end1=end1
        self.end2=end2
    def test(self,points):
        tester=[0<=x*self.function[1]-y*self.function[0]-self.function[2] for x,y in points]
        if False in tester:
            return False
        else:
            return True
    def show(self):
        return self.end1,self.end2


class Poly:
    def __init__(self,points):
        self.points=points
        self.edges=[Edge(a,b) for a in points for b in points if Edge(a,b).test(points) and a!=b]
    def filterout(self,points):
        return {k for k in points if False not in [edge.test([k]) for edge in self.edges]}

In [ ]:
import os,pickle

class Fully_Autonomous_Drone_Model():
    """
    Situation 3
    """
    def __init__(self,Map,Address,T_speed,D_Range,D_speed,D_number,Driver_Delay):
        self.Address=Address
        self.T_speed=T_speed
        self.D_Range=D_Range
        self.D_speed=D_speed
        self.D_number=D_number
        self.Travel=(complex(np.inf,np.inf),[])
        
        Bounds=Poly(self.Address)
        
        k=Progbar(target=2*(len(Address))*len(range(0,D_Range,2)))
        
        self.Drone_Drop=[Nodes_Near_LEGACY(Addres,16,3,Map.NodeList,k,D_Range) for Addres in Address[1:]]
        self.Drone_Drop=[Bounds.filterout(self.Drone_Drop[i]) if len(Bounds.filterout(self.Drone_Drop[i]))>0 else {Address[i+1]} for i in range(len(self.Drone_Drop))]
        
        Paths=dict()
        print("Building All Paths before Runtime")
        k1=Progbar(target=sum([len(list(itertools.product(*comb))) for comb in itertools.combinations([set(Address[0:1])]+self.Drone_Drop,2)]))
        
        for startstop in itertools.combinations([set(Address[0:1])]+self.Drone_Drop,2):
            for start,stop in itertools.product(*startstop):
                    k1.add(1)

                    Map.start=Map.NodeList[Nearest([start],Map.NodeList)]
                    Map.end  =Map.NodeList[Nearest([stop],Map.NodeList)]
                    x=astar.solve(Map)[0]
                    xc=[]
                    for eac in range(len(x)-1):
                        for (X,Y) in [(X,Y) for X in list(range(x[eac].Position[0], x[eac+1].Position[0], 
                                                    [k if k==-1 else 1 for k in [np.sign(x[eac+1].Position[0]-x[eac].Position[0])]][0]
                                                     ))+[x[eac+1].Position[0]] 
                                     for Y in list(range(x[eac].Position[1], x[eac+1].Position[1], 
                                                    [k if k==-1 else 1 for k in [np.sign(x[eac+1].Position[1]-x[eac].Position[1])]][0]
                                                     ))+[x[eac+1].Position[1]]
                                    ][:-1]:
                            xc.append((X,Y))
                    x=xc
                    
                    Paths[(start,stop)]=x
                    x=x.copy()
                    x.reverse()
                    Paths[(stop,start)]=x

        

        
        k2=Progbar(target=len(list(itertools.product(*self.Drone_Drop))))
        
        for path in itertools.product(*self.Drone_Drop):

            k2.add(1)
            path=tuple(Address[0:1])+path

            
            
            
            PathGrid=np.array([[Paths[(begin,end)] if begin!=end else [begin] for begin in path] for end in path])
            
            
            self.pat=Traditional_Model(Map,path,T_speed,Driver_Delay/2,k1,k2,False,Path_Grid=PathGrid)
            self.pat.Travel=[complex(0,0),[0]+self.pat.Travel[1]]
            
            self.pat.Travel[0]+=complex(len(self.pat.Path_Grid[self.pat.Travel[1][0]][self.pat.Travel[1][1]]),0)
            
            full=self.pat.Path_Grid[self.pat.Travel[1][0]][self.pat.Travel[1][1]]
            
            for each in range(len(self.pat.Travel[1])-2):
                truck_Can_Move=True
                package_delivered=False
                phi=cmath.phase(-complex(path[self.pat.Travel[1][each+1]][0],
                                        path[self.pat.Travel[1][each+1]][1]
                                       )+complex(
                                        Address[self.pat.Travel[1][each+1]][0],
                                        Address[self.pat.Travel[1][each+1]][1]
                                       ))
                direction=cmath.rect(D_speed/T_speed,phi)
                drone=complex(path[self.pat.Travel[1][each+1]][0],path[self.pat.Travel[1][each+1]][1])+direction
                truck=complex(path[self.pat.Travel[1][each+1]][0],path[self.pat.Travel[1][each+1]][1])
                house=complex(Address[self.pat.Travel[1][each+1]][0],Address[self.pat.Travel[1][each+1]][1])
                trip=self.pat.Path_Grid[self.pat.Travel[1][each+1]][self.pat.Travel[1][each+2]]
                i=0
                
                while cmath.polar(drone-truck)[0]>D_speed/T_speed and not package_delivered:
#                     print(truck,drone)
                    if package_delivered:
                        direction=cmath.rect(D_speed/T_speed,cmath.phase(truck-drone))
                    if cmath.polar(drone-house)[0]<D_speed/T_speed and not package_delivered:
                        drone=house
                        package_delivered=True
                        self.pat.Travel[0]+=complex(0,Driver_Delay/2)
                    else:
                        drone+=direction
                    if cmath.polar(drone-truck)[0]>D_Range:
                        truck_Can_Move=False
                    if truck_Can_Move:
                        i+=1
                        if i>len(trip)-2:
                            truck_Can_Move=False
                            continue
                        truck=complex(trip[i][0],trip[i][1])
                        self.pat.Travel[0]+=complex(1,0)
                    else:
                        self.pat.Travel[0]+=complex(0,1)
                    if i<len(trip):
                        full.append(trip[i])
                    else:
                        full.append(trip[-1])
                self.pat.Travel[0]+=complex(0,cmath.polar(drone-truck)[0])
                self.pat.Travel[0]+=complex(len(trip[i:]),0)
                full.extend(trip[i:])
                    
                
            
            if self.pat.Travel[0].real+self.pat.Travel[0].imag<self.Travel[0].real+self.Travel[0].imag:
                self.Travel=self.pat.Travel
                self.path=self.pat
                self.points=path
                self.full=full
        print("\n-")
                
    def fullPath(self):
        return self.full

In [ ]:
class Instance():
    """
    Compile 3 models together for a given list of addresses
    """
    def __init__(self,Map,Address,T_Speed,D_Speed,Driver_Delay,D_Number,D_Range):
        self.Traditional=Traditional_Model(Map,Address,T_Speed,Driver_Delay,verbose=True)
        self.Drone=Drone_Model_FAA_Comp(Map,Address,T_Speed,D_Range,D_Speed,D_Number,Driver_Delay)
        self.NDrone=Fully_Autonomous_Drone_Model(Map,Address,T_Speed,D_Range,D_Speed,D_Number,Driver_Delay)
        print("")
        print(self.Traditional.Travel[0],self.Drone.Travel[0])#,self.NDrone.Travel[0])

# Simulation

In [ ]:
# DF=pd.DataFrame(columns=["Traditional","Drone","Relaxed"])
# i=0
while True:
    i+=1
    Address=Get_Address(im,15)
    I=Instance(maze,
         Address,
         T_Speed=30,
         D_Speed=15,
         Driver_Delay=32,
         D_Number=1,
         D_Range=32)
    clear_output()
    df=pd.DataFrame({"Traditional":[I.Traditional.Travel[1]],"Drone":[I.Drone.Travel[1]],"Relaxed":[I.NDrone.Travel[1]]},index=[i])
    DF=DF.append(df)
    with open("Range Updated Order.pkl","wb") as f:
        pickle.dump(DF,f)
    

# Given Route Planning

In [ ]:
Address=[(515, 1604), (581, 1745), (251, 1350), (665, 439), (312, 697)] 

I=Instance(maze,
         Address,
         T_Speed=30,
         D_Speed=15,
         Driver_Delay=32,
         D_Number=1,
         D_Range=32)

Traditional_Travel_Time=I.Traditional.Travel[0].real+I.Traditional.Travel[0].imag
Drone_Model_FAA_Comp   =I.Drone.Travel[0].real+I.Drone.Travel[0].imag
Drone_Model_Full_Auto  =I.NDrone.Travel[0].real+I.NDrone.Travel[0].imag

if Traditional_Travel_Time<Drone_Model_FAA_Comp and
   Traditional_Travel_Time<Drone_Model_Full_Auto:
        print(I.Traditional.fullPath())

elif Drone_Model_FAA_Comp<Traditional_Travel_Time and
     Drone_Model_FAA_Comp<Drone_Model_Full_Auto:
        print(I.Drone.fullPath())

else:
    print(I.NDrone.fullPath())

In [ ]:
DF